In [1]:
!pip install setfit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.2/75.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Load your dataset
data = pd.read_csv("/content/shreeallahujesus.csv")
print("Original class distribution:")
print(data["class_value"].value_counts())

# Separate the data by class
class_0 = data[data["class_value"] == 0]
class_1 = data[data["class_value"] == 1]

# Downsample class 0 to match the size of class 1 (65 samples)
class_0_downsampled = class_0.sample(n=74, random_state=42)

# Concatenate the downsampled class 0 with class 1 to create a balanced dataset
balanced_data = pd.concat([class_0_downsampled, class_1], axis=0)
balanced_data = balanced_data.rename(columns={"class_value": "label", "concatenated_text": "text"})


# Shuffle the dataset to avoid ordering bias
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Check the new class distribution
print("\nBalanced class distribution:")
print(balanced_data["label"].value_counts())

# Split into train and test sets (80-20 split)
train_data, eval_data  = train_test_split(balanced_data, stratify=balanced_data["label"],test_size=0.2, random_state=42)
train_dataset = Dataset.from_pandas(train_data)
eval_dataset = Dataset.from_pandas(eval_data)

print(f"Training samples: {train_data.shape}, Test samples: {eval_data.shape}")

Original class distribution:
class_value
0    114
1     74
Name: count, dtype: int64

Balanced class distribution:
label
1    74
0    74
Name: count, dtype: int64
Training samples: (118, 2), Test samples: (30, 2)


In [ ]:
train_dataset

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 118
})

In [3]:
train_dataset = train_dataset.remove_columns("__index_level_0__")
eval_dataset = eval_dataset.remove_columns("__index_level_0__")


In [4]:
from setfit import SetFitModel, SetFitTrainer,TrainingArguments
from sentence_transformers.losses import CosineSimilarityLoss

# Load a SetFit model from Hugging Face
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-MiniLM-L6-v2")

# Check if the model is loaded correctly
print(f"Loaded model: {model}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Loaded model: <setfit.modeling.SetFitModel object at 0x783108dd0700>


In [5]:
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_iterations=20, # Number of text pairs to generate for contrastive learning
    num_epochs=3 # Number of epochs to use for contrastive learning
)

<ipython-input-5-e62d0b2b45dc>:1: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(


Map:   0%|          | 0/118 [00:00<?, ? examples/s]

In [6]:
trainer.train()


***** Running training *****
  Num unique pairs = 4720
  Batch size = 16
  Num epochs = 3
/usr/local/lib/python3.10/dist-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,0.297000
50,0.255200
100,0.253400
150,0.245700
200,0.210400
250,0.084000
300,0.013000
350,0.006100
400,0.004700
450,0.003900


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [7]:
metrics = trainer.evaluate(eval_dataset)

metrics

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
***** Running evaluation *****


{'accuracy': 0.7}

In [8]:
input = ["I havent been feeling the best these days but I am optimistic."]
output = model(input)
output

tensor([0])

In [ ]:
output

tensor([1])

In [9]:
model.save_pretrained("setfit-finetuned-model")

In [10]:
!zip -r /content/setfit-finetuned-model.zip /content/setfit-finetuned-model


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  adding: content/setfit-finetuned-model/ (stored 0%)
  adding: content/setfit-finetuned-model/model.safetensors (deflated 8%)
  adding: content/setfit-finetuned-model/special_tokens_map.json (deflated 80%)
  adding: content/setfit-finetuned-model/tokenizer_config.json (deflated 75%)
  adding: content/setfit-finetuned-model/sentence_bert_config.json (deflated 4%)
  adding: content/setfit-finetuned-model/vocab.txt (deflated 53%)
  adding: content/setfit-finetuned-model/1_Pooling/ (stored 0%)
  adding: content/setfit-finetuned-model/1_Pooling/config.json (deflated 57%)
  adding: content/setfit-finetuned-model/model_head.pkl (deflated 9%)
  adding: content/setfit-finetuned-model/config_setfit.json (deflated 6%)
  adding: content/setfit-finetuned-model/README.md (deflated 72%)
  adding: content/setfit-finetuned-model/.ipynb_checkpoints/ (stored 0%)
  adding: content/setfit-finetuned-model/config_sentence_transformers.json (deflated 36%)
  adding: content/setfit-finetuned-model/modules.json

In [11]:
model = SetFitModel.from_pretrained("/content/setfit-finetuned-model")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
model.predict(["I didnt even have the motivation to get out of bed these days. I dont even feel happy"])


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tensor([1])

In [19]:
file_path = "/content/shreeallahujesus.csv"
data = pd.read_csv(file_path)
data.columns

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['concatenated_text', 'class_value'], dtype='object')